In [19]:
#import packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

In [20]:
#import data
df_crime = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/MergedData_2017-2021.csv')
df_covid_ward = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                              , sheet_name = 'Total Cases by Ward', engine='openpyxl')
df_covid_death = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                               , sheet_name = 'Lives Lost by Ward', engine='openpyxl')
df_sick_police = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/daily_count_police_out_due_to_covid.csv')

In [21]:
#delete columns not needed from crime data
del df_crime['REPORT_DAT']
del df_crime['BLOCK']
#del df_crime['YBLOCK']
#del df_crime['XBLOCK']
#del df_crime['LATITUDE']
#del df_crime['LONGITUDE']
del df_crime['END_DATE']

df_crime = df_crime.rename(columns = {'START_DATE':'date'})

df_crime['date'] = pd.to_datetime(df_crime['date']).dt.date
df_crime['date'] = pd.to_datetime(df_crime['date'])
df_crime.dtypes
df_crime

,SHIFT,METHOD,OFFENSE,XBLOCK,YBLOCK,WARD,DISTRICT,LATITUDE,LONGITUDE,date
0,MIDNIGHT,OTHERS,THEFT F/AUTO,403249.170000,138982.570000,5.0,5.0,38.918707,-76.962533,2017-10-28
1,MIDNIGHT,KNIFE,ASSAULT W/DANGEROUS WEAPON,399214.790000,137753.750000,6.0,1.0,38.907643,-77.009053,2017-10-28
2,MIDNIGHT,OTHERS,THEFT F/AUTO,398532.800000,137052.350000,6.0,1.0,38.901323,-77.016914,2017-10-28
3,DAY,OTHERS,THEFT F/AUTO,398374.890000,135044.830000,6.0,1.0,38.883239,-77.018730,2017-10-27
4,DAY,OTHERS,THEFT/OTHER,400246.540000,139229.040000,5.0,5.0,38.920933,-76.997157,2017-10-28
...,...,...,...,...,...,...,...,...,...,...
151064,DAY,OTHERS,BURGLARY,402085.390000,138351.140000,5.0,5.0,38.913022,-76.975955,2021-02-04
151065,DAY,OTHERS,THEFT F/AUTO,402855.420321,134087.377978,7.0,6.0,38.874610,-76.967094,2021-02-04
151066,DAY,OTHERS,THEFT F/AUTO,400720.210000,142604.330000,5.0,4.0,38.951338,-76.991691,2021-02-03
151067,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,401535.270000,131621.840000,8.0,7.0,38.852403,-76.982313,2021-02-04


In [22]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151069 entries, 0 to 151068
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SHIFT      151069 non-null  object        
 1   METHOD     151069 non-null  object        
 2   OFFENSE    151069 non-null  object        
 3   XBLOCK     151069 non-null  float64       
 4   YBLOCK     151069 non-null  float64       
 5   WARD       147528 non-null  float64       
 6   DISTRICT   150926 non-null  float64       
 7   LATITUDE   151069 non-null  float64       
 8   LONGITUDE  151069 non-null  float64       
 9   date       151064 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 11.5+ MB


In [23]:
df_crime.isna().sum()

SHIFT           0
METHOD          0
OFFENSE         0
XBLOCK          0
YBLOCK          0
WARD         3541
DISTRICT      143
LATITUDE        0
LONGITUDE       0
date            5
dtype: int64

In [24]:
#drop records with null records
df_crime = df_crime.dropna(axis=0)
print(df_crime.isna().sum())
print(df_crime.shape)

SHIFT        0
METHOD       0
OFFENSE      0
XBLOCK       0
YBLOCK       0
WARD         0
DISTRICT     0
LATITUDE     0
LONGITUDE    0
date         0
dtype: int64
(147385, 10)


In [25]:
#covid case data
df_covid_ward = df_covid_ward.transpose()
df_covid_ward.reset_index(drop = True, inplace= True)
df_covid_ward.columns=('date', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'unknown')
df_covid_ward = df_covid_ward.iloc[1: , :]
df_covid_ward['date'] = pd.to_datetime(df_covid_ward['date'])
df_covid_ward['total_cases'] = (df_covid_ward['ward_1'] + df_covid_ward['ward_2'] + df_covid_ward['ward_3'] + df_covid_ward['ward_4'] + df_covid_ward['ward_5'] + df_covid_ward['ward_6'] +df_covid_ward['ward_7'] + df_covid_ward['ward_8'] +df_covid_ward['unknown'])
df_covid_ward['daily_cases'] = df_covid_ward['total_cases'].diff(1)

del df_covid_ward['ward_1']
del df_covid_ward['ward_2']
del df_covid_ward['ward_3']
del df_covid_ward['ward_4']
del df_covid_ward['ward_5']
del df_covid_ward['ward_6']
del df_covid_ward['ward_7']
del df_covid_ward['ward_8']
del df_covid_ward['unknown']

df_covid_ward.fillna(0, inplace=True)

df_covid_ward

,date,total_cases,daily_cases
1,2020-03-31,586,0
2,2020-04-01,653,67
3,2020-04-02,757,104
4,2020-04-03,902,145
5,2020-04-04,998,96
...,...,...,...
536,2021-11-01,64541,71
537,2021-11-02,64602,61
538,2021-11-03,64708,106
539,2021-11-04,64799,91


In [26]:
#death data
df_covid_death = df_covid_death.transpose()
df_covid_death.reset_index(drop = True, inplace= True)
df_covid_death.columns=('date', 'all', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'Experienced Homelessness', 'unknown', 'na')
df_covid_death = df_covid_death.iloc[1: , :]
df_covid_death['date'] = pd.to_datetime(df_covid_death['date'])
df_covid_death['daily_death'] = df_covid_death['all'].diff(1)

del df_covid_death['ward_1']
del df_covid_death['ward_2']
del df_covid_death['ward_3']
del df_covid_death['ward_4']
del df_covid_death['ward_5']
del df_covid_death['ward_6']
del df_covid_death['ward_7']
del df_covid_death['ward_8']
del df_covid_death['unknown']
del df_covid_death['na']
del df_covid_death['Experienced Homelessness']
del df_covid_death['all']

df_covid_death.fillna(0, inplace=True)

df_covid_death

,date,daily_death
1,2020-04-19,0
2,2020-04-20,7
3,2020-04-21,15
4,2020-04-22,12
5,2020-04-23,14
...,...,...
515,2021-11-01,2
516,2021-11-02,0
517,2021-11-03,0
518,2021-11-04,0


In [27]:
#sick police
df_sick_police = df_sick_police.rename(columns = {'DATE_REPORTED':'date'})
df_sick_police['date'] = pd.to_datetime(df_sick_police['date'])

df_sick_police.fillna(0, inplace=True)

df_sick_police

,date,OUT_POSITIVE_QUARANTINE_PSMPD
0,2020-03-07,0.0
1,2020-03-08,0.0
2,2020-03-09,0.0
3,2020-03-10,0.0
4,2020-03-11,0.0
...,...,...
589,2021-10-17,42.0
590,2021-10-18,43.0
591,2021-10-19,43.0
592,2021-10-20,43.0


In [28]:
#merge the dataframes together
merged = df_crime.merge(df_covid_ward, how='left', on = 'date').merge(df_covid_death, how='left', on = 'date').merge(df_sick_police, how='left', on = 'date')
merged

,SHIFT,METHOD,OFFENSE,XBLOCK,YBLOCK,WARD,DISTRICT,LATITUDE,LONGITUDE,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD
0,MIDNIGHT,OTHERS,THEFT F/AUTO,403249.170000,138982.570000,5.0,5.0,38.918707,-76.962533,2017-10-28,NaN,NaN,NaN,NaN
1,MIDNIGHT,KNIFE,ASSAULT W/DANGEROUS WEAPON,399214.790000,137753.750000,6.0,1.0,38.907643,-77.009053,2017-10-28,NaN,NaN,NaN,NaN
2,MIDNIGHT,OTHERS,THEFT F/AUTO,398532.800000,137052.350000,6.0,1.0,38.901323,-77.016914,2017-10-28,NaN,NaN,NaN,NaN
3,DAY,OTHERS,THEFT F/AUTO,398374.890000,135044.830000,6.0,1.0,38.883239,-77.018730,2017-10-27,NaN,NaN,NaN,NaN
4,DAY,OTHERS,THEFT/OTHER,400246.540000,139229.040000,5.0,5.0,38.920933,-76.997157,2017-10-28,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147380,DAY,OTHERS,BURGLARY,402085.390000,138351.140000,5.0,5.0,38.913022,-76.975955,2021-02-04,37634.0,269.0,8.0,104.0
147381,DAY,OTHERS,THEFT F/AUTO,402855.420321,134087.377978,7.0,6.0,38.874610,-76.967094,2021-02-04,37634.0,269.0,8.0,104.0
147382,DAY,OTHERS,THEFT F/AUTO,400720.210000,142604.330000,5.0,4.0,38.951338,-76.991691,2021-02-03,37365.0,166.0,6.0,106.0
147383,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,401535.270000,131621.840000,8.0,7.0,38.852403,-76.982313,2021-02-04,37634.0,269.0,8.0,104.0


In [29]:
#getting weekday dummies
merged['weekday'] = merged['date'].dt.dayofweek
dummy0 = pd.get_dummies(merged['weekday'])
dummy0.columns = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']
merged = pd.concat([merged, dummy0] , axis = 1)
del merged['weekday']

#get month dummies
merged['month'] = merged['date'].dt.month
dummy1 = pd.get_dummies(merged['month'])
dummy1.columns = ['JAN', 'FEB','MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
                 'OCT', 'NOV', 'DEC']              
merged = pd.concat([merged, dummy1] , axis = 1)
del merged['month']

#ward dummies
dummy2 = pd.get_dummies(merged['WARD'])
dummy2.columns = ['ward_1', 'ward_2','ward_3', 'ward_4', 'ward_5', 'ward_6', 'ward_7', 'ward_8']              
merged = pd.concat([merged, dummy2] , axis = 1)

merged.columns

Index(['SHIFT', 'METHOD', 'OFFENSE', 'XBLOCK', 'YBLOCK', 'WARD', 'DISTRICT',
       'LATITUDE', 'LONGITUDE', 'date', 'total_cases', 'daily_cases',
       'daily_death', 'OUT_POSITIVE_QUARANTINE_PSMPD', 'MO', 'TU', 'WE', 'TH',
       'FR', 'SA', 'SU', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ward_1', 'ward_2', 'ward_3',
       'ward_4', 'ward_5', 'ward_6', 'ward_7', 'ward_8'],
      dtype='object')

In [30]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147385 entries, 0 to 147384
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   SHIFT                          147385 non-null  object        
 1   METHOD                         147385 non-null  object        
 2   OFFENSE                        147385 non-null  object        
 3   XBLOCK                         147385 non-null  float64       
 4   YBLOCK                         147385 non-null  float64       
 5   WARD                           147385 non-null  float64       
 6   DISTRICT                       147385 non-null  float64       
 7   LATITUDE                       147385 non-null  float64       
 8   LONGITUDE                      147385 non-null  float64       
 9   date                           147385 non-null  datetime64[ns]
 10  total_cases                    38189 non-null   float64       
 11  

In [31]:
#replace nan with 0
merged.fillna(0, inplace=True)

merged

,SHIFT,METHOD,OFFENSE,XBLOCK,YBLOCK,WARD,DISTRICT,LATITUDE,LONGITUDE,date,...,NOV,DEC,ward_1,ward_2,ward_3,ward_4,ward_5,ward_6,ward_7,ward_8
0,MIDNIGHT,OTHERS,THEFT F/AUTO,403249.170000,138982.570000,5.0,5.0,38.918707,-76.962533,2017-10-28,...,0,0,0,0,0,0,1,0,0,0
1,MIDNIGHT,KNIFE,ASSAULT W/DANGEROUS WEAPON,399214.790000,137753.750000,6.0,1.0,38.907643,-77.009053,2017-10-28,...,0,0,0,0,0,0,0,1,0,0
2,MIDNIGHT,OTHERS,THEFT F/AUTO,398532.800000,137052.350000,6.0,1.0,38.901323,-77.016914,2017-10-28,...,0,0,0,0,0,0,0,1,0,0
3,DAY,OTHERS,THEFT F/AUTO,398374.890000,135044.830000,6.0,1.0,38.883239,-77.018730,2017-10-27,...,0,0,0,0,0,0,0,1,0,0
4,DAY,OTHERS,THEFT/OTHER,400246.540000,139229.040000,5.0,5.0,38.920933,-76.997157,2017-10-28,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147380,DAY,OTHERS,BURGLARY,402085.390000,138351.140000,5.0,5.0,38.913022,-76.975955,2021-02-04,...,0,0,0,0,0,0,1,0,0,0
147381,DAY,OTHERS,THEFT F/AUTO,402855.420321,134087.377978,7.0,6.0,38.874610,-76.967094,2021-02-04,...,0,0,0,0,0,0,0,0,1,0
147382,DAY,OTHERS,THEFT F/AUTO,400720.210000,142604.330000,5.0,4.0,38.951338,-76.991691,2021-02-03,...,0,0,0,0,0,0,1,0,0,0
147383,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,401535.270000,131621.840000,8.0,7.0,38.852403,-76.982313,2021-02-04,...,0,0,0,0,0,0,0,0,0,1


In [32]:
count = merged.groupby(['OFFENSE'], as_index=False).agg(
{
    'date':'count',
})
count

,OFFENSE,date
0,ARSON,33
1,ASSAULT W/DANGEROUS WEAPON,7824
2,BURGLARY,6386
3,HOMICIDE,804
4,MOTOR VEHICLE THEFT,12785
5,ROBBERY,9722
6,SEX ABUSE,1061
7,THEFT F/AUTO,46406
8,THEFT/OTHER,62364


In [33]:
merged.loc[merged['daily_cases'] != 0].count()

SHIFT                            37420
METHOD                           37420
OFFENSE                          37420
XBLOCK                           37420
YBLOCK                           37420
WARD                             37420
DISTRICT                         37420
LATITUDE                         37420
LONGITUDE                        37420
date                             37420
total_cases                      37420
daily_cases                      37420
daily_death                      37420
OUT_POSITIVE_QUARANTINE_PSMPD    37420
MO                               37420
TU                               37420
WE                               37420
TH                               37420
FR                               37420
SA                               37420
SU                               37420
JAN                              37420
FEB                              37420
MAR                              37420
APR                              37420
MAY                      

In [34]:
merged.isna().sum()

SHIFT                            0
METHOD                           0
OFFENSE                          0
XBLOCK                           0
YBLOCK                           0
WARD                             0
DISTRICT                         0
LATITUDE                         0
LONGITUDE                        0
date                             0
total_cases                      0
daily_cases                      0
daily_death                      0
OUT_POSITIVE_QUARANTINE_PSMPD    0
MO                               0
TU                               0
WE                               0
TH                               0
FR                               0
SA                               0
SU                               0
JAN                              0
FEB                              0
MAR                              0
APR                              0
MAY                              0
JUN                              0
JUL                              0
AUG                 

In [35]:
#split into precovid/during covid
merged.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/merged.csv')
